In [1]:
import scipy
import numpy as np
from airlift.envs import AirliftEnv, AirliftWorldGenerator, ActionHelper, generators
from airlift.envs import StaticCargoGenerator, DynamicCargoGenerator
import airlift
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
class Dijkstra:
    def __init__(self, cargo_generator, airport_generator):
        self.env = AirliftEnv(AirliftWorldGenerator(cargo_generator=cargo_generator,airport_generator=airport_generator))
        self.obs = self.env.reset()
        self.planes = self.obs.keys()
        self.rewards = []

    def reset(self):
        self.obs = self.env.reset()
        self.planes = self.obs.keys()
        self.rewards = []

    def print_obs(self):
        return self.obs

    def get_rewards(self):
        return self.rewards

    def optimal_route(self, plane):
        cargo = self.obs[plane]['globalstate']["active_cargo"]
        g = self.obs[plane]["globalstate"]["route_map"][0]
        curr_location = self.obs[plane]["current_airport"]

        min_cost = 1000
        path_cargo = []
        path_deliver = []
        cargo_id = -1
        for i in cargo:
            length1, path1 = nx.single_source_dijkstra(g, curr_location, target=i.location, weight="time")
            length2, path2 = nx.single_source_dijkstra(g, i.location, target=i.destination, weight="time")
            total = length1 + length2
            if total <= i.hard_deadline:
                path = path1 + path2[1:]
                print(path)
                cost = 0
                for j in range(len(path) - 1):
                    cost += g[path[j]][path[j+1]]["cost"]
                if cost <= min_cost:
                    min_cost = cost
                    path_cargo = path1
                    path_deliver = path2
                    cargo_id = i.id
        return cargo_id, path_cargo, path_deliver

    def heur_test(self, plane):
        tmp = self.obs[plane]['globalstate']["active_cargo"]
        avaliable_cargo = {}
        for i in tmp:
            avaliable_cargo[i.id] = i
        g = self.obs[plane]["globalstate"]["route_map"][0]
        curr_location = self.obs[plane]["current_airport"]

        c1, path, time = self.get_closest(curr_location, avaliable_cargo, g)
        cargo = {}
        cargo[c1] = avaliable_cargo[c1]
        del avaliable_cargo[c1]

        while len(cargo) <= self.obs[plane][weight]

        time = min_l
        path = path_c1
        cargo = {}
        cargo[c1] = avaliable_cargo[c1]
        del avaliable_cargo[c1]

        curr_location = path[-1]

        min_l = 1000
        c2 = -1
        path_c2 = []
        for i in avaliable_cargo.keys():
            length, p = nx.single_source_dijkstra(g, curr_location, target=avaliable_cargo[i].location, weight="time")

            if length == min_l:
                if avaliable_cargo[i].soft_deadline < avaliable_cargo[c2].soft_deadline:
                    c2 = i
                    path_c2 = path
            elif length < min_l:
                c2 = i
                path_c2 = path
                min_l = length

    def get_closest(self, curr_location, avaliable_cargo, g):
        min_l = 1000
        c = -1
        path_c = []
        for i in avaliable_cargo.keys():
            length, path = nx.single_source_dijkstra(g, curr_location, target=avaliable_cargo[i].location, weight="time")

            if length == min_l:
                if avaliable_cargo[i].soft_deadline < avaliable_cargo[c].soft_deadline:
                    c = i
                    path_c = path
            elif length < min_l:
                c = i
                path_c = path
                min_l = length
        return c, path_c, min_l



    def route_test(self, plane):
        tmp = self.obs[plane]['globalstate']["active_cargo"]
        cargo = {}
        for i in tmp:
            cargo[i.id] = i
        g = self.obs[plane]["globalstate"]["route_map"][0]
        curr_location = self.obs[plane]["current_airport"]

        min_l = 1000
        c1 = -1
        path_c1 = []
        for i in cargo.keys():
            length, path = nx.single_source_dijkstra(g, curr_location, target=cargo[i].location, weight="time")

            if length == min_l:
                if cargo[i].soft_deadline < cargo[c1].soft_deadline:
                    c1 = i
                    path_c1 = path
            elif length < min_l:
                c1 = i
                path_c1 = path
                min_l = length

        curr_location = path_c1[-1]
        min_l2 = 1000
        c2 = -1
        path_c2 = []
        for i in cargo.keys():
            if i != c1:
                length, path = nx.single_source_dijkstra(g, curr_location, target=cargo[i].location, weight="time")

                if length == min_l2:
                    if cargo[i].soft_deadline < cargo[c1].soft_deadline:
                        c2 = i
                        path_c2 = path
                elif length < min_l2:
                    c2 = i
                    path_c2 = path
                    min_l2 = length

        len_total = min_l + min_l2






    def init_step(self):
        self.flight_plan = {}
        for plane in self.planes:
            cargo_id, path_cargo, path_deliver = self.optimal_route(plane)
            self.flight_plan[plane] = {"cargo_id": cargo_id, "path_cargo": path_cargo[1:], "path_deliver": path_deliver[1:]}
        print(self.flight_plan)

    def step(self):
        actions = {}
        for i in self.flight_plan.keys():
            o = self.obs[i]
            actions[i] = {}
            if self.flight_plan[i]["path_cargo"]:
                # We have just landed at the airport with the cargo we need to pick up
                if o["current_airport"] == self.flight_plan[i]["path_cargo"][-1]:
                    actions[i]["process"] = 1
                    actions[i]["cargo_to_load"] = [self.flight_plan[i]["cargo_id"]]
                    actions[i]["destination"] = o["current_airport"]
                    actions[i]["cargo_to_unload"] = []
                    self.flight_plan[i]["path_cargo"] = self.flight_plan[i]["path_cargo"][1:]

                else:
                    actions[i]["process"] = 0
                    actions[i]["cargo_to_load"] = []
                    actions[i]["destination"] = self.flight_plan[i]["path_cargo"][0]
                    actions[i]["cargo_to_unload"] = []

            else:
                if o["current_airport"] == self.flight_plan[i]["path_deliver"][-1]:
                    actions[i]["process"] = 1
                    actions[i]["cargo_to_load"] = []
                    actions[i]["destination"] = o["current_airport"]
                    actions[i]["cargo_to_unload"] = [self.flight_plan[i]["cargo_id"]]
                    self.flight_plan[i]["path_deliver"] = self.flight_plan[i]["path_deliver"][1:]

                else:
                    actions[i]["process"] = 0
                    actions[i]["cargo_to_load"] = []
                    actions[i]["destination"] = self.flight_plan[i]["path_deliver"][0]
                    actions[i]["cargo_to_unload"] = []

        self.obs, rewards, dones, infos = self.env.step(actions)
        self.rewards.append(rewards)


In [3]:
cargo_generator=generators.cargo_generators.StaticCargoGenerator(3)
airport_generator=generators.airport_generators.RandomAirportGenerator(10, num_pick_up_airports=5, num_drop_off_airports=4)
dij = Dijkstra(cargo_generator, airport_generator)

In [4]:
dij.print_obs()

{'a_0': {'cargo_onboard': [],
  'state': <PlaneState.READY_FOR_TAKEOFF: 3>,
  'plane_type': 0,
  'current_weight': 0,
  'max_weight': 5,
  'available_routes': [1, 3, 4, 5, 6, 7, 8, 9, 10],
  'next_action': {'process': 0,
   'cargo_to_load': [],
   'cargo_to_unload': [],
   'destination': 0},
  'current_airport': 2,
  'cargo_at_current_airport': [],
  'destination': 0,
  'globalstate': {'route_map': {0: <networkx.classes.digraph.DiGraph at 0x2852507baf0>},
   'plane_types': [PlaneTypeObservation(id=0, max_weight=5)],
   'agents': {'a_0': {'cargo_onboard': [],
     'state': <PlaneState.READY_FOR_TAKEOFF: 3>,
     'plane_type': 0,
     'current_weight': 0,
     'max_weight': 5,
     'available_routes': [1, 3, 4, 5, 6, 7, 8, 9, 10],
     'next_action': {'process': 0,
      'cargo_to_load': [],
      'cargo_to_unload': [],
      'destination': 0},
     'current_airport': 2,
     'cargo_at_current_airport': [],
     'destination': 0}},
   'active_cargo': [CargoObservation(id=0, location=7, d

In [5]:
dij.init_step()

[2, 7, 3]
[2, 10, 2]
[2, 9, 4]
{'a_0': {'cargo_id': 1, 'path_cargo': [10], 'path_deliver': [2]}}


In [27]:
dij.step()
dij.print_obs()

IndexError: list index out of range

In [28]:
dij.get_rewards()

[{'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': -0.0009561},
 {'a_0': 0},
 {'a_0': 0},
 {'a_0': 0},
 {'a_0': -0.0009561},
 {'a_0': -1.0009561},
 {'a_0': -3.0009561},
 {'a_0': -3.0009561},
 {'a_0': -3.0009561},
 {'a_0': -3.0009561},
 {'a_0': -3.0009561},
 {'a_0': -12.0009561},
 {'a_0': -2},
 {'a_0': -20}]